# Sequence model
A sequence model is an older type of model that exposes raw sentances to train on. 



In [1]:
from tensorflow.keras import layers
import tensorflow as tf
#tf.config.set_visible_devices([], 'GPU')
#import os
#os.environ['KMP_DUPLICATE_LIB_OK']='True'


2023-07-07 18:31:56.528903: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 18:31:57.202615: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
max_length = 600
max_tokens = 20000
text_vec = layers.TextVectorization(max_tokens=max_tokens, output_mode = "int", output_sequence_length=max_length)

2023-07-07 18:31:57.823181: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 18:31:57.843425: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-07 18:31:57.843610: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
from tensorflow import keras
train_ds = keras.utils.text_dataset_from_directory("/home/kaspar/Documents/code/data/aclImdb/train", batch_size=32)
val_ds = keras.utils.text_dataset_from_directory("/home/kaspar/Documents/code/data/aclImdb/val",batch_size=32)
test_ds = keras.utils.text_dataset_from_directory("/home/kaspar/Documents/code/data/aclImdb/test",batch_size=32)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [4]:
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vec.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_val_ds = val_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)
int_test_ds = test_ds.map(lambda x,y: (text_vec(x),y), num_parallel_calls=4)

In [5]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)


In [7]:
callbacks = keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras", save_best_only=True)
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")


NameError: name 'model' is not defined

Okay this is very very slow without a GPU! Unfortuntely Windows being the awful operating system that it is does not support graphical acceleration...

# Embeddings
Up untiul now I have been using one-hot encoding which converts a word into a vector. However these vectors have no knowledge of other vacabularly. This is almost as if they have their own seperate orthogonal space. Emdings allow us to map the connections between words. 

In [6]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256) # 256 is the dimensionality

In [7]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirection  (None, 64)                73984     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [9]:
callbacks = keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras")
model.fit(int_train_ds.cache(), validation_data=int_val_ds.cache(), epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 66s 106ms/step - loss: 0.3581 - accuracy: 0.8630 - val_loss: 0.3391 - val_accuracy: 0.8626
Epoch 2/10
625/625 [==============================] - 37s 59ms/step - loss: 0.2838 - accuracy: 0.8994 - val_loss: 0.3331 - val_accuracy: 0.8668
Epoch 3/10
625/625 [==============================] - 36s 58ms/step - loss: 0.2353 - accuracy: 0.9201 - val_loss: 0.3444 - val_accuracy: 0.8690
Epoch 4/10
625/625 [==============================] - 37s 59ms/step - loss: 0.2091 - accuracy: 0.9319 - val_loss: 0.3630 - val_accuracy: 0.8676
Epoch 5/10
625/625 [==============================] - 37s 59ms/step - loss: 0.1834 - accuracy: 0.9406 - val_loss: 0.3780 - val_accuracy: 0.8668
Epoch 6/10
625/625 [==============================] - 37s 59ms/step - loss: 0.1624 - accuracy: 0.9481 - val_loss: 0.3916 - val_accuracy: 0.8696
Epoch 7/10
625/625 [==============================] - 37s 59ms/step - loss: 0.1415 - accuracy: 0.9575 - val_loss: 0.4103 - val_accuracy

The above code truncates the sequence after 600 words so this is not yet as accurate as the bigrams method. A masking would work better to achieve slighty more accuracy. 

In [10]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs) ### IMPORTANT ###
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

## Note on GPU
It appears that the use of GPU prevents us using options within the call back such as save best only = True. This is a bit of an issue. However looking ast the limiings we went from 5 minutes per step to just under 1  minute when using Cuda. Thus even if we have to retrain with a limited number of epochs it is still worthwhile using the GPU to perform tasks. 